In [ ]:
from huggingface_hub import login
login('')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from datasets import load_dataset, concatenate_datasets
import random
from datasets import Dataset, DatasetDict


dataset = load_dataset("Angelectronic/ViMMRC")
dataset

Generating train split:   0%|          | 0/1936 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/551 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/296 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 1936
    })
    test: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 551
    })
    dev: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 296
    })
})

In [3]:
!pip install accelerate peft bitsandbytes transformers trl evaluate sacrebleu sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring-parser 0.15
    Uninstalling docstring-parser-0.15:
      Successfully uninstalled docstring-parser-0.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud

In [4]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("Angelectronic/llama3-8b-ViMMRC")
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3-8b-bnb-4bit")
model = PeftModel.from_pretrained(base_model, "Angelectronic/llama3-8b-ViMMRC")
tokenizer = AutoTokenizer.from_pretrained("Angelectronic/llama3-8b-ViMMRC")

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
END_TURN_TOKEN = '<end_of_turn>'
BOS_TOKEN = tokenizer.bos_token

def formatting_prompts_func(examples):
    texts = []
    for i in range(len(examples['question'])):
        option_prompt = ""
        for j in range(len(examples['options'][i])):
            if j == 0:
                option_prompt += "A. " + examples['options'][i][j] + "\n"
            elif j == 1:
                option_prompt += "B. " + examples['options'][i][j] + "\n"
            elif j == 2:
                option_prompt += "C. " + examples['options'][i][j] + "\n"
            elif j == 3:
                option_prompt += "D. " + examples['options'][i][j] + "\n"

        chat = [
            {"role": "user", "content": "Use the following reference to answer the given multiple choice-question:\nReference: " + examples['context'][i] + "\nQuestion: " + examples['question'][i] + "\n" + option_prompt},
            {"role": "system", "content": examples['answers'][i]}
        ]
        texts.append(tokenizer.apply_chat_template(chat, tokenize=False) + EOS_TOKEN)

    texts = [text.split(BOS_TOKEN)[-1] for text in texts]
    return {"text": texts}
pass
dataset['train'] = dataset['train'].map(formatting_prompts_func, batched=True)
dataset['test'] = dataset['test'].map(formatting_prompts_func, batched=True)

collator = DataCollatorForCompletionOnlyLM(instruction_template="<|im_start|>user", response_template="<|im_start|>system", tokenizer=tokenizer, mlm=False)

print(dataset['train'][-1]['text'])

2024-07-10 13:23:53.525578: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 13:23:53.525689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 13:23:53.733600: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/1936 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


Map:   0%|          | 0/551 [00:00<?, ? examples/s]

<|im_start|>user
Use the following reference to answer the given multiple choice-question:
Reference: Khách đến xã Trịnh Tường, huyện Bát Xát, tỉnh Lào Cai  sẽ không khỏi ngỡ ngàng thấy một dòng mương ngoằn ngoèo vắt ngang những đồi cao. Dân bản gọi dòng mương ấy là con nước ông Lìn. Để thay đổi tập quán làm lúa nương, ông Phàn Phù Lìn, người Dao ở thôn Phìn Ngan đã lần mò cả tháng trong rừng tìm nguồn nước. Nhưng tìm được nguồn nước rồi, mọi người vẫn không tin có thể dẫn nước về. Ông cùng vợ con đào suốt một năm trời được gần bốn cây số mương xuyên đồi dẫn nước từ rừng già về thôn, trồng một héc ta lúa nước để bà con tin. Rồi ông vận động mọi người cùng mở rộng con mương, vỡ thêm đất hoang trồng lúa.

Con nước nhỏ đã làm thay đổi tập quán canh tác và cuộc sống của trên 50 hộ trong thôn. Những nương lúa quanh năm khát nước được thay dần bằng ruộng bậc thang. Những giống lúa lai cao sản được ông Lìn đưa về vận động bà con trồng cấy, nhờ vậy mà cả thôn không còn hộ đói. Từ khi nước được

In [6]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

END_TURN_TOKEN = '<|im_end|>'
BOS_TOKEN = tokenizer.bos_token
EOS_TOKEN = tokenizer.eos_token

labels = [dataset['test'][i]['answers'] for i in range(len(dataset['test']))]
preds = []
preds1 = []
# FastLanguageModel.for_inference(model)
for i in tqdm(range(len(dataset['test']))):
    option_prompt = ""
    for j in range(len(dataset['test'][i]['options'])):
        if j == 0:
            option_prompt += "A. " + dataset['test'][i]['options'][j] + "\n"
        elif j == 1:
            option_prompt += "B. " + dataset['test'][i]['options'][j] + "\n"
        elif j == 2:
            option_prompt += "C. " + dataset['test'][i]['options'][j] + "\n"
        elif j == 3:
            option_prompt += "D. " + dataset['test'][i]['options'][j] + "\n"

    chat = [
        {"role": "user", "content": "Use the following reference to answer the given multiple choice-question:\nReference: " + dataset['test'][i]['context'] + "\nQuestion: " + dataset['test'][i]['question'] + "\n" + option_prompt},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    prompt += "<|im_start|>system"
    prompt = prompt.split(BOS_TOKEN)[-1]
    inputs = tokenizer(
        prompt,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=256, use_cache = True, do_sample = False, eos_token_id=model.config.eos_token_id, pad_token_id=tokenizer.eos_token_id)

#     logits = outputs.logits
#     token_ids = torch.argmax(logits, dim=-1)
    
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=False)
    completion = prediction.split(END_TURN_TOKEN)[-2].strip()
    preds.append(completion[-1])

print(accuracy_score(labels, preds))

  0%|          | 0/551 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 551/551 [36:44<00:00,  4.00s/it]

0.8493647912885662


In [7]:
print(preds)

['B', 'B', 'B', 'A', 'A', 'A', 'B', 'D', 'B', 'C', 'B', 'A', 'A', 'B', 'C', 'C', 'C', 'C', 'B', 'A', 'A', 'A', 'C', 'D', 'C', 'B', 'B', 'C', 'C', 'A', 'B', 'C', 'A', 'D', 'C', 'A', 'D', 'A', 'B', 'D', 'D', 'A', 'D', 'D', 'B', 'A', 'A', 'A', 'D', 'A', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'A', 'B', 'A', 'A', 'C', 'A', 'B', 'A', 'C', 'C', 'A', 'B', 'C', 'A', 'B', 'A', 'C', 'D', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'D', 'D', 'B', 'C', 'D', 'C', 'D', 'B', 'C', 'A', 'C', 'D', 'A', 'A', 'B', 'C', 'C', 'C', 'C', 'B', 'A', 'A', 'D', 'C', 'C', 'A', 'B', 'D', 'C', 'A', 'A', 'B', 'B', 'D', 'C', 'A', 'C', 'A', 'C', 'D', 'A', 'A', 'D', 'B', 'B', 'C', 'B', 'B', 'D', 'A', 'C', 'C', 'C', 'A', 'B', 'B', 'B', 'A', 'C', 'B', 'B', 'A', 'A', 'B', 'A', 'C', 'C', 'A', 'B', 'A', 'C', 'A', 'A', 'A', 'A', 'A', 'A', 'C', 'A', 'C', 'B', 'D', 'D', 'A', 'A', 'C', 'D', 'A', 'B', 'B', 'B', 'D', 'A', 'B', 'A', 'B', 'D', 'C', 'D', 'C', 'B', 'D', 'A', 'A', 'C', 'B', 'B', 'C', 'C', 'B', 'C', 'D', 'C', 'B', 'A', 'D', 'A', 'D',